# ***Comparison b/w RNN & SVR***

In [ ]:
# ------------------------------------------------------------------------------
# Setup tflite runtime
# ------------------------------------------------------------------------------
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!sudo apt-get update
!sudo apt-get install python3-tflite-runtime

# ------------------------------------------------------------------------------
# Clone the repo and add `src` to path
# ------------------------------------------------------------------------------
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install
!git clone https://github.com/epfl-lasa/sliding-ds-control.git
import sys
sys.path.insert(1, "/content/sliding-ds-control/src")

# ------------------------------------------------------------------------------
# Download Helper and Data Files
# ------------------------------------------------------------------------------
output = "helper_and_data_files.zip"
use_gdrive = False
if use_gdrive:
    # Download the zip file from Google Drive
    import gdown
    url = "https://drive.google.com/uc?id=1JJ9pFGGlpWIVFgDudmcquVuTdtGvASvq"
    gdown.download(url, output, quiet=False)
else:
    # Download via Git-LFS
    !ln -s /content/sliding-ds-control/compliant_bumper/rnn_training/helper_and_data_files.zip .

# Extract zip file
import zipfile
with zipfile.ZipFile(output, "r") as zip_ref:
    zip_ref.extractall(".")


# ------------------------------------------------------------------------------
# Imports
# ------------------------------------------------------------------------------
from loadmat import loadmat
import numpy as np
from scipy.signal import decimate

%matplotlib notebook
import matplotlib.pyplot as plt

from timeit import default_timer as timer

import prediction_model

In [ ]:
test_files  = ["calibration_struct{:d}.mat".format(i) for i in [2, 4, 6, 7, 5]]
test_database = "calibration_struct2"
n_history = 5
# ------------------------------------------------------------------------------
# Lambda functions for data extractions
# ------------------------------------------------------------------------------
get_X = lambda tmp: np.vstack((
    tmp["Fx_measured"],
    tmp["Fy_measured"],
    tmp["Fz_measured"],
    tmp["Mx_measured"],
    tmp["My_measured"],
    tmp["Mz_measured"],
)).T

get_y = lambda tmp: np.vstack((
    tmp["Fx_applied"],
    tmp["Fy_applied"],
    tmp["Mz_applied"],
)).T

# ------------------------------------------------------------------------------
# Load Data
# ------------------------------------------------------------------------------
X_test = {}
y_test = {}
for filename in test_files:
    print(f"Loading {filename} ...")
    data = loadmat(filename)
    data = data["training_data_struct"]
    X_temp = np.empty((0,6))
    y_temp = np.empty((0,3))
    for test in data.keys():
        X_temp = np.vstack((X_temp, get_X(data[test])))
        y_temp = np.vstack((y_temp, get_y(data[test])))
    X_test[filename[:-4]] = X_temp
    y_test[filename[:-4]] = y_temp
# ------------------------------------------------------------------------------
# Downsample from 400Hz to 200 Hz (realworld control frequency)
# ------------------------------------------------------------------------------
for filename in test_files:
    X_test[filename[:-4]] = decimate(X_test[filename[:-4]], 2, axis=0)
    y_test[filename[:-4]] = decimate(y_test[filename[:-4]], 2, axis=0)

In [ ]:
y_ground = y_test[test_database]

# ------------------------------------------------------------------------------
# Comparison b/w RNN & SVR
# ------------------------------------------------------------------------------
model_nn = prediction_model.BumperModel(model_type="nn")
for j in range(X_test[test_database].shape[0]):
    x = np.array([X_test[test_database][j,:]])
    if j == 0:
        y_est_nn = [model_nn.predict(x)]
    else:
        y_est_nn = np.append(y_est_nn, [model_nn.predict(x)], axis=0)

model_svr = prediction_model.BumperModel(model_type="svr")
for j in range(X_test[test_database].shape[0]):
    x = np.array([X_test[test_database][j,:]])
    if j == 0:
        y_est_svr = [model_svr.predict(x)]
    else:
        y_est_svr = np.append(y_est_svr, [model_svr.predict(x)], axis=0)

# ------------------------------------------------------------------------------
# Plots
# ------------------------------------------------------------------------------
out_name = ["Fx", "Fy", "Mz"]
colors = {
    "ground": "#d95f02",
    "rnn": "#1b9e77",
    "svr": "#7570b3",
}

plt.figure(test_database)
for i in range(len(out_name)):
    plt.subplot(3,1,i+1)
    plt.plot(y_ground[:, i], linewidth=3, color=colors["ground"])
    plt.plot(y_est_nn[:, i], color=colors["rnn"])
    plt.plot(y_est_svr[:, i], color=colors["svr"])
    if out_name[i] is "Fx":
        plt.ylabel("Fx [N]")
    elif out_name[i] is "Fy":
        plt.ylabel("Fy [N]")
    elif out_name[i] is "Mz":
        plt.ylabel("Mz [N-m]")
    plt.tight_layout()
    plt.xlim((100000, 130000))
    plt.xticks([])
    plt.show()
plt.legend(["Ground Truth", "RNN", "SVR"])

In [ ]:
# ------------------------------------------------------------------------------
# Timing Comparison
# ------------------------------------------------------------------------------
n = 1000
x_single = np.array([X_test[test_database][1,:]])

model_nn = prediction_model.BumperModel(model_type="nn")
start = timer()
[model_nn.predict(x_single) for _ in range(n)]
end = timer()
ellapsed = end - start
print("---- RNN ----")
print("Total Elapsed Time for {:d} calls = {:.5f} s".format(n, ellapsed))
print("Avg. Elapsed Time per call = {:.5f} s".format(ellapsed/n))

model_svr = prediction_model.BumperModel(model_type="svr")
start = timer()
[model_svr.predict(x_single) for _ in range(n)]
end = timer()
ellapsed = end - start
print("---- SVR ----")
print("Total Elapsed Time for {:d} calls = {:.5f} s".format(n, ellapsed))
print("Avg. Elapsed Time per call = {:.5f} s".format(ellapsed/n))